In [5]:
import numpy as np
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 52.52,  # Latitude for Berlin
    "longitude": 13.405,  # Longitude for Berlin
    "start_date": "2018-12-24",  # Start date
    "end_date": "2023-12-11",  # End date
    "hourly": "temperature_2m"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
    start=pd.to_datetime(hourly.Time(), unit="s"),
    end=pd.to_datetime(hourly.TimeEnd(), unit="s"),
    freq=pd.Timedelta(seconds=hourly.Interval()),
    inclusive="left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data=hourly_data)
hourly_dataframe.to_csv("temperatures.csv", index=True)
print(hourly_dataframe)


Coordinates 52.5483283996582°E 13.407821655273438°N
Elevation 37.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                     date  temperature_2m
0     2018-12-24 00:00:00           4.815
1     2018-12-24 01:00:00           4.665
2     2018-12-24 02:00:00           3.915
3     2018-12-24 03:00:00           3.365
4     2018-12-24 04:00:00           2.965
...                   ...             ...
43531 2023-12-11 19:00:00           7.165
43532 2023-12-11 20:00:00           7.115
43533 2023-12-11 21:00:00           7.115
43534 2023-12-11 22:00:00           6.715
43535 2023-12-11 23:00:00           6.565

[43536 rows x 2 columns]


In [3]:
from models.DAX import DAX_quantile_regression, DAX_baseline
from models.energy import energy_quantile_regression
from functions import get_energy, get_DAX, merge_submissions, check_submission
from functions.prepare_data import split_time
from functions.evaluation import evaluate_horizon
from tqdm import tqdm
from functions import evaluate_dax
from functions import naive_ensemble
import pandas as pd
#import minmaxscaler
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, date, timedelta

In [25]:
energydata = get_energy.get()
energydata.head()

  0%|          | 0/260 [00:00<?, ?it/s]/Users/stephantimpe/PycharmProjects/probabilistic-forecasting-challenge/functions/get_energy.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 260/260 [00:27<00:00,  9.48it/s]


,gesamt,weekday
date_time,,
2018-12-24 00:00:00,42.02925,0
2018-12-24 01:00:00,39.61025,0
2018-12-24 02:00:00,39.13875,0
2018-12-24 03:00:00,39.42100,0
2018-12-24 04:00:00,40.74775,0


In [49]:
import pandas as pd

# Assuming 'energydata' is your energy DataFrame and 'hourly_dataframe' is your weather DataFrame
# Also assuming both have a column 'date' or 'timestamp' that you want to set as index



hourly_dataframe



,temperature_2m
date,
2018-12-24 00:00:00,4.815
2018-12-24 01:00:00,4.665
2018-12-24 02:00:00,3.915
2018-12-24 03:00:00,3.365
2018-12-24 04:00:00,2.965
...,...
2023-12-11 19:00:00,7.165
2023-12-11 20:00:00,7.115
2023-12-11 21:00:00,7.115


In [50]:
# Join the DataFrames on their indices
df_combined = energydata.join(hourly_dataframe, how='inner')

# Now df_combined has both energy and weather data aligned by datetime index

In [51]:
df_combined

,gesamt,weekday,temperature_2m
2018-12-24 00:00:00,42.02925,0,4.815
2018-12-24 01:00:00,39.61025,0,4.665
2018-12-24 02:00:00,39.13875,0,3.915
2018-12-24 03:00:00,39.42100,0,3.365
2018-12-24 04:00:00,40.74775,0,2.965
...,...,...,...
2023-12-11 19:00:00,65.65500,0,7.165
2023-12-11 20:00:00,62.07900,0,7.115
2023-12-11 21:00:00,58.18600,0,7.115
2023-12-11 22:00:00,54.64750,0,6.715


In [53]:
energy_quantile_regression(df_combined)

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).